In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import chi2_contingency

In [2]:
# beolvassuk a fájlt -->df
df = pd.read_csv('output.tsv', sep='\t')

# töröljük azokat a sorokat, amelyekben nincs ige: kopula?
df = df[df['pv'].notna()]

# ilyen a táblázat
df.head(20)

,@id,pv,lemma,pvv,mood,n,PV,INF,HKM,Vva,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
0,f01s1w7,0,megy,0+megy,i,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,f01s1w6,0,kirándul,0+kirándul,n,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,f01s2w3,0,megy,0+megy,i,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,f01s2w8,0,van,0+van,i,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,f01s3w4,0,látszik,0+látszik,i,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,f01s3w13,0,árulkodik,0+árulkodik,i,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,f01s4w5,0,jelent,0+jelent,i,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,f01s5w7,haza,visz,haza+visz,i,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,f01s6w1,0,van,0+van,i,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,f01s6w13,0,van,0+van,i,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# kigyűjtjük az előforduló igekötőket +0
pvs=df['pv'].unique()

# kigyűjtjük az eseteket
Cases = df.columns.values[6:]

In [4]:
# létrehozunk egy üres táblázatot, az igekötők a sorok (+total), az esetek az oszlopok (+n)
pv1 = pd.DataFrame(columns=df.columns.values[5:])
pv1 = pv1.append(pd.DataFrame(index=["total"]))
pv1 = pv1.append(pd.DataFrame(index=pvs))


In [5]:
# kitöltjük a táblázatot

# először az összes esetet megszámoljuk
pv1.loc['total']['n']=df['n'].sum()

# ezután megszámoljuk, hogy az egyes esetek hányszor fordulnak elő
# ha egy ige mellett két azonos esetű bővítmény is van, csak egynek számoljuk
for j in Cases:
    pv1.loc['total'][j]=df[df[j] > 0]['n'].sum()

# ezután kitöltjük az igekötős oszlopokat
for i in pvs:
    # először összegezzük, hogy hányszor fordul elő az igekötő
    pv1.loc[i]['n']=df[df['pv'] == i]['n'].sum()
    # majd megnézzük, hogy az adott igekötő mellet hányszor fordulnak elő az egyes esetek
    for j in Cases:
        pv1.loc[i][j]=df[(df[j] > 0) & (df['pv'] == i)]['n'].sum()

In [6]:
pv1.to_csv('pvCount.tsv', sep='\t', encoding='utf-8')
pv1

,n,PV,INF,HKM,Vva,nom,acc,dat,BA,BAN,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
total,165686,17449,16900,13218,2832,79219,52127,8530,5711,16876,...,13161,3792,9,551,11,2451,9147,12572,1327,935
0,116144,2845,15587,9555,2167,59470,33208,7041,3665,12682,...,9840,2785,4,343,7,1438,6353,9076,1242,684
haza,745,214,12,19,10,101,37,3,7,12,...,37,3,0,2,0,3,39,42,0,3
le,2677,605,102,95,40,895,971,29,67,158,...,223,307,0,13,0,16,188,143,2,8
be,4001,1307,42,272,46,1601,1501,131,805,327,...,204,152,0,8,1,28,223,224,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
odább,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
partra,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kétségbe,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fent,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Itt számoljuk ki a X2 értékeket
# először létrehozzuk a táblázatot, ugyanolyan, mint az előző, csak nincs total és n
pvX2 = pd.DataFrame(columns=df.columns.values[6:])
pvX2 = pvX2.append(pd.DataFrame(index=pvs))
pvP = pd.DataFrame(columns=df.columns.values[6:])
pvP = pvP.append(pd.DataFrame(index=pvs))

# sorbavesszük az igekötőket és az eseteket
for pv in pvs:
    for case in Cases:
        # kivesszük az előző számlálós táblázatból a releváns értékeket
        x = pv1.loc['total']['n'] 
        xs = pv1.loc[pv]['n']
        xo = pv1.loc['total'][case]
        xso = pv1.loc[pv][case]
        
        c3 = xso
        b3 = xs-xso
        c2 = xo-xso
        b2 = x-b3-c2-c3
        
#        eb2=(b2+b3)*(b2+c2)/x
#        eb3=(b2+b3)*(b3+c3)/x
#        ec2=(c2+c3)*(b2+c2)/x
#        ec3=(c2+c3)*(b3+c3)/x
        
        data = [[b2,b3],[c2,c3]]

        stat, p, dof, expected = chi2_contingency(data)
        
        if expected[0][0]<5 or expected[0][1]<5 or expected[1][0]<5 or expected[1][1]<5:
            # ha nincs elég várt adat, akkor nem lehet X2-t számolni
            pvX2.loc[pv][case]=-1
            pvP.loc[pv][case]=-1
        else:
            # egyébként
            pvX2.loc[pv][case]=stat
            pvP.loc[pv][case]=p < 0.01

In [19]:
pvX2.to_csv('pvX2.tsv', sep='\t', encoding='utf-8', decimal=',')
pvP.to_csv('pvP.tsv', sep='\t', encoding='utf-8', decimal=',')
pvX2.head(20)


,PV,INF,HKM,Vva,nom,acc,dat,BA,BAN,BÓL,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
0,26923.1,4396.77,32.7216,56.3368,1789.51,1482.5,663.885,98.7553,228.283,476.486,...,148.353,20.5564,-1,15.8731,-1,154.482,1.8856,28.333,351.195,4.04489
haza,260.963,59.3384,29.2899,0.400529,350.564,242.394,33.5442,13.3894,59.2114,1.81135,...,8.66515,11.071,-1,-1,-1,5.23294,0.0686062,3.78473,5.07194,-1
le,419.307,120.58,72.0923,0.624527,224.9,28.9757,91.229,7.0015,54.0972,1.05671,...,0.504477,1021.07,-1,1.48251,-1,13.9028,11.4795,19.2513,17.144,2.95357
be,2129.69,373.757,7.60522,7.30323,99.591,69.4074,29.0975,3419.58,17.9287,35.3272,...,44.9727,41.1353,-1,1.78453,-1,16.5489,0.0128463,22.8476,32.0776,3.76187
el,2889.29,157.514,21.7135,6.01523,278.178,42.7107,88.6536,0.0149206,95.5591,34.9115,...,143.779,39.9662,-1,0.689625,-1,977.178,6.98792,0.312602,84.4981,0.532287
meg,4785.98,993.723,0.16006,26.1315,105.204,1220.16,62.2069,384.142,7.29753e-05,89.3324,...,522.11,47.5774,-1,0.179241,-1,98.487,0.894414,6.40889,115.09,1.29912
át,790.905,124.935,45.4396,0.370236,50.174,150.741,16.3865,45.7826,8.43125,0.270375,...,104.792,6.20412,-1,-1,-1,0.918146,0.422353,8.42236,242.979,0.105384
tovább,117.381,14.8372,6.92128,-1,79.3444,23.4483,0.630475,0.207026,2.90544,-1,...,9.92904,-1,-1,-1,-1,-1,0.132655,0.731504,-1,-1
ki,4070.3,458.499,19.2279,4.71543,209.823,204.11,104.458,55.4997,40.174,7412.12,...,1.94982,11.14,-1,1.30374,-1,41.7353,0.346722,48.803,29.9233,8.09541
neki,8.7339,0.0487808,4.56213,-1,22.1223,39.0049,744.126,-1,5.49403,-1,...,6.18185,-1,-1,-1,-1,-1,2.40863,0.812631,-1,-1


In [20]:
# Itt számoljuk ki a Cramer's V értékeket
# először létrehozzuk a táblázatot, ugyanolyan, mint az előző, csak nincs total és n
pvCV = pd.DataFrame(columns=df.columns.values[6:])
pvCV = pvCV.append(pd.DataFrame(index=pvs))

# sorbavesszük az igekötőket és az eseteket
for pv in pvs:
    for case in Cases:
        n = pv1.loc[pv]['n']
        if pvX2.loc[pv][case] > 0:
            pvCV.loc[pv][case] = math.sqrt(pvX2.loc[pv][case] / n)

In [21]:
pvCV.to_csv('pvCV.tsv', sep='\t', encoding='utf-8', decimal=',')
pvCV

,PV,INF,HKM,Vva,nom,acc,dat,BA,BAN,BÓL,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
0,0.481465,0.194567,0.0167849,0.0220241,0.124128,0.112979,0.0756046,0.0291596,0.0443341,0.0640511,...,0.0357396,0.0133038,NaN,0.0116905,NaN,0.0364703,0.00402927,0.0156188,0.054989,0.0059014
haza,0.591849,0.282221,0.198281,0.0231867,0.685971,0.570404,0.212193,0.134061,0.281919,0.0493087,...,0.107847,0.121903,NaN,NaN,NaN,0.0838098,0.00959629,0.0712754,0.0825105,NaN
le,0.395769,0.212233,0.164104,0.015274,0.289848,0.104038,0.184605,0.0511412,0.142155,0.019868,...,0.0137277,0.617594,NaN,0.0235329,NaN,0.0720654,0.0654844,0.0848019,0.0800262,0.0332162
be,0.729581,0.30564,0.0435985,0.0427241,0.157771,0.13171,0.0852793,0.92449,0.0669407,0.0939659,...,0.106021,0.101397,NaN,0.0211192,NaN,0.0643133,0.00179186,0.0755676,0.08954,0.0306632
el,0.538572,0.12575,0.0466889,0.0245739,0.167113,0.0654812,0.0943402,0.00122389,0.0979455,0.0592015,...,0.120142,0.0633425,NaN,0.00832061,NaN,0.31321,0.0264864,0.00560202,0.0921026,0.00731007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
odább,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
partra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kétségbe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Itt számoljuk ki az előfordulási gyakoriságot
# először létrehozzuk a táblázatot, ugyanolyan, mint az előző, csak nincs n
pvFreq = pd.DataFrame(columns=df.columns.values[6:])
pvFreq = pvFreq.append(pd.DataFrame(index=["total"]))
pvFreq = pvFreq.append(pd.DataFrame(index=pvs))

for case in Cases:
    n = pv1.loc['total']['n']
    pvFreq.loc['total'][case] = pv1.loc['total'][case] / n
        
# sorbavesszük az igekötőket és az eseteket
for pv in pvs:
    for case in Cases:
        n = pv1.loc[pv]['n']
        pvFreq.loc[pv][case] = pv1.loc[pv][case] / n

In [12]:
pvFreq.to_csv('pvFreq.tsv', sep='\t', encoding='utf-8', decimal=',')
pvFreq

,PV,INF,HKM,Vva,nom,acc,dat,BA,BAN,BÓL,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
total,0.105314,0.102,0.0797774,0.0170926,0.478127,0.314613,0.0514829,0.0344688,0.101855,0.0205389,...,0.0794334,0.0228867,5.43196e-05,0.00332557,6.63906e-05,0.014793,0.0552068,0.0758785,0.00800913,0.0056432
0,0.0244955,0.134204,0.0822686,0.0186579,0.512037,0.285921,0.060623,0.0315557,0.109192,0.0155669,...,0.0847224,0.0239789,3.444e-05,0.00295323,6.027e-05,0.0123812,0.0546993,0.0781444,0.0106936,0.00588924
haza,0.287248,0.0161074,0.0255034,0.0134228,0.13557,0.0496644,0.00402685,0.00939597,0.0161074,0.0281879,...,0.0496644,0.00402685,0,0.00268456,0,0.00402685,0.052349,0.0563758,0,0.00402685
le,0.225999,0.0381024,0.0354875,0.0149421,0.334329,0.362719,0.010833,0.025028,0.0590213,0.017557,...,0.0833022,0.114681,0,0.00485618,0,0.00597684,0.0702279,0.053418,0.000747105,0.00298842
be,0.326668,0.0104974,0.067983,0.0114971,0.40015,0.375156,0.0327418,0.2012,0.0817296,0.00724819,...,0.0509873,0.0379905,0,0.0019995,0.000249938,0.00699825,0.0557361,0.055986,0,0.00324919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
odább,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
partra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kétségbe,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fent,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Itt számoljuk ki a Cramer's V értékek erősségét
# először létrehozzuk a táblázatot, ugyanolyan, mint az előző, csak nincs total és n
pvCV01 = pd.DataFrame(columns=df.columns.values[6:])
pvCV01 = pvCV01.append(pd.DataFrame(index=pvs))

# sorbavesszük az igekötőket és az eseteket
for pv in pvs:
    for case in Cases:
        n = pv1.loc[pv]['n']
        if pvX2.loc[pv][case] > 0:
            w = math.sqrt(pvX2.loc[pv][case] / n)
            if w < 0.1 :
                pvCV01.loc[pv][case] = 0
            elif w < 0.3 :
                pvCV01.loc[pv][case] = 1
            elif w < 0.5 :
                pvCV01.loc[pv][case] = 2
            else :
                pvCV01.loc[pv][case] = 3
            

In [8]:
pvCV01.to_csv('pvCV03.tsv', sep='\t', encoding='utf-8', decimal=',')
pvCV01

,PV,INF,HKM,Vva,nom,acc,dat,BA,BAN,BÓL,...,RA,RÓL,STUL,SZOR,TT,TÓL,UL,VAL,VÁ,ÉRT
0,2,1,0,0,1,1,0,0,0,0,...,0,0,NaN,0,NaN,0,0,0,0,0
haza,3,1,1,0,3,3,1,1,1,0,...,1,1,NaN,NaN,NaN,0,0,0,0,NaN
le,2,1,1,0,1,1,1,0,1,0,...,0,3,NaN,0,NaN,0,0,0,0,0
be,3,2,0,0,1,1,0,3,0,0,...,1,1,NaN,0,NaN,0,0,0,0,0
el,3,1,0,0,1,0,0,0,0,0,...,1,0,NaN,0,NaN,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
odább,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
partra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kétségbe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
